# build toy model and save it to pickle

In [3]:
import pickle
import boto3

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Train the model

iris = load_iris()
data, labels = iris.data, iris.target
training_data, test_data, training_labels, test_labels = train_test_split(data, labels)

model = LogisticRegression(solver="lbfgs", multi_class="multinomial")
model.fit(training_data, training_labels)
accuracy = model.score(test_data, test_labels)
print("accuracy: {:.2f}".format(accuracy))

# Upload the model

pickle.dump(model, open("toy_model.pkl", "wb"))
s3 = boto3.client("s3")
s3.upload_file("toy_model.pkl", "cortex-ee16ff5ab7", "dev/toy_model.pkl")

accuracy: 0.95


In [43]:
s3.upload_file("lgbm_model.pkl", "cortex-ee16ff5ab7", "dev/lgbm_model.pkl")

## Write Predictor Class

In [42]:
%%writefile predictor.py

import os
import pickle

labels = ["True", "False"]


class PythonPredictor:
    def __init__(self, config):
        """
        if os.environ.get("AWS_ACCESS_KEY_ID"):
            s3 = boto3.client("s3")  # client will use your credentials if available
        else:
            s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))  # anonymous client

        s3.download_file(config["bucket"], config["key"], "/tmp/model.pkl")
        """
        self.model = pickle.load(open("lgbm_model.pkl", "rb"))

    def predict(self, payload):
   
        label_id = self.model.predict([payload['data']])[0]
        return labels[label_id]

Overwriting predictor.py


In [33]:
import pandas as pd

df = pd.read_csv('../../assets/dataset/home_db.csv', index_col=None)
df.shape
df.head()

y = df.loc[:,'home_purchase']
X = df.loc[:,:'YOUNGEST_DEPENDENT_CHILD_BIRTH_YEAR']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [41]:
payload = {'data': list(X_test.loc[0].values)}
rf = PythonPredictor(None).model
rf.predict([payload['data']])

array([False])

## Deployment Configuration

In [44]:
%%writefile cortex.yaml
- name: gamification-model
  kind: RealtimeAPI
  predictor:
    type: python
    path: predictor.py
    config:
      bucket: cortex-ee16ff5ab7
      key: dev/lgbm_model.pkl

Overwriting cortex.yaml


## Deployment Requirements

In [46]:
%%writefile requirements.txt
pandas
scikit-learn
lightgbm

Overwriting requirements.txt


## Sample Query Payload

In [52]:
import json
json.dumps(payload)

'{"data": [0.0, 55.02699309780146, 325000.0, 1981.0, 676.9948191352805, 0.193144071416786, 1000.0, NaN, 1.0, 3.0, 1000.0, 68387.0, NaN, NaN, 83197.0, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, 4650.0, 16249.999999999998, 0.0, 1.0, NaN, NaN, NaN, NaN, 1015.0, NaN, 501.56916060815, 138826.0, 67.0, 84892.0, 119167.0, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, 21.0, NaN, NaN]}'

In [53]:
%%writefile sample.json
{"data": [0.0, 55.02699309780146, 325000.0, 1981.0, 676.9948191352805, 0.193144071416786, 1000.0, NaN, 1.0, 3.0, 1000.0, 68387.0, NaN, NaN, 83197.0, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, 4650.0, 16249.999999999998, 0.0, 1.0, NaN, NaN, NaN, NaN, 1015.0, NaN, 501.56916060815, 138826.0, 67.0, 84892.0, 119167.0, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, 21.0, NaN, NaN]}

Overwriting sample.json


## Deploy Model
deploy the model in terminal

In [12]:
# !cortex deploy
# !cortex get 
# !cortex get recommender

## Query Deployed Model

In [55]:
!curl https://t0erni03x7.execute-api.us-west-1.amazonaws.com/gamification-model -X POST -H "Content-Type: application/json" -d @sample.json

True